<a href="https://colab.research.google.com/github/TrelisResearch/trelis-collab-chat/blob/main/Trelis_Chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *About Trelis Chat*

A Chat Assistant built on Llama 2.

Find Trelis on [HuggingFace](https://huggingface.co/Trelis).

# Setup and Installation

- You can run Trelis Chat on a free Google Colab Notebook.
- Save a copy of this notebook: Go to File -> Save a copy in Drive. (optional, but needed if you want to make changes).
- Go to the menu -> Runtime -> Change Runtime Type - Select GPU (T4).
- Then go to Runtime -> Run all.
- You'll be asked to connect to your Google Drive to download the language model. This is optional and you can delete that step below, but it will make the notebook slower to load next time.
- It can take about 5-10 mins for the notebook to start.

!!! Trelis has zero access to your data when you use this notebook. All of your data remains within your Google Drive and Google's computers !!!

In [1]:
# Set the runtime to cpu or gpu. Leave as gpu for Google Colab.
runtime = "gpu"  # OR "cpu"

if runtime == "cpu":
    runtimeFlag = "cpu"
elif runtime == "gpu":
    runtimeFlag = "cuda:0"
else:
    print("Invalid runtime. Please set it to either 'cpu' or 'gpu'.")
    runtimeFlag = None

cache_dir = None # by default, don't set a cache directory
print("Runtime flag is:", runtimeFlag)

Runtime flag is: cuda:0


In [2]:
### Select the language model and model_basename
model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = "gptq_model-4bit-128g"


#### Google Drive Mounting (optional)
- Allows you to download the model to Google Drive for faster startup next time.

In [3]:
import os
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
## Allow the model to be saved to Google Drive for faster startup next time

# This is the path to the Google Drive folder.
drive_path = "/content/drive"

# This is the path where you want to store your cache.
cache_dir_path = os.path.join(drive_path, "My Drive/huggingface_cache")

# Check if the Google Drive folder exists. If it does, use it as the cache_dir.
# If not, set cache_dir to None to use the default Hugging Face cache location.
if os.path.exists(drive_path):
    cache_dir = cache_dir_path
    os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists
else:
    cache_dir = None

print(cache_dir)

/content/drive/My Drive/huggingface_cache


### Install

In [5]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [6]:
## some of these are not needed, like bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U auto-gptq, pdfminer.six

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


### Import

In [7]:
import transformers
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

## Load Model
If you have connected to Google Drive, the model will load from there (unless this is your first time connecting, in which case the model will be saved to Drive).
- Takes about 6 mins first time around.
- Takes about 30s 2nd time onwards with Google Drive.

In [8]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

os.environ["SAFETENSORS_FAST_GPU"] = "1"

model_name_or_path = "TheBloke/Llama-2-7b-Chat-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None,
        cache_dir=cache_dir)

# """
# To download from a specific branch, use the revision parameter, as in this example:

# model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
#         revision="gptq-4bit-32g-actorder_True",
#         model_basename=model_basename,
#         use_safetensors=True,
#         trust_remote_code=True,
#         device="cuda:0",
#         quantize_config=None)
# """

In [9]:
print(model.config.max_length)

4096


## Set up the Tokenizer

In [10]:
# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, cache_dir=cache_dir, use_fast=True) # will use the Rust fast tokenizer if available

In [122]:
print("BOS token:", tokenizer.bos_token)
print("EOS token:", tokenizer.eos_token)

BOS token: <s>
EOS token: </s>


# Inference (Chat Style)


In [38]:
from IPython.display import display, HTML, clear_output, Markdown
import textwrap
import ipywidgets as widgets
import re, time
from google.colab import files
from pdfminer.high_level import extract_text
import io

In [147]:
DEFAULT_SYSTEM_PROMPT = 'You are a helpful assistant that provides accurate and concise responses. Respond in markdown.'
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# max_doc_length = 50
max_doc_length = int(0.5 * model.config.max_length)  # adjust according to your model's maximum length
max_doc_words = int(0.75*max_doc_length)

In [152]:
#Just show an error when limit is reached.

def generate_response(dialogs, temperature=0.6, top_p=0.9, logprobs=False):
    torch.cuda.empty_cache()
    max_prompt_len = int(0.85 * model.config.max_length)
    max_gen_len = int(0.10 * max_prompt_len)

    prompt_tokens = []
    for dialog in dialogs:
        if dialog[0]["role"] != "system":
            dialog = [
                {
                    "role": "system",
                    "content": DEFAULT_SYSTEM_PROMPT,
                }
            ] + dialog
        dialog_tokens = [tokenizer(
            f"{B_INST} {B_SYS}{(dialog[0]['content']).strip()}{E_SYS}{(dialog[1]['content']).strip()} {E_INST}",
            return_tensors="pt",
            add_special_tokens=True
        ).input_ids.to(runtimeFlag)]
        for i in range(2, len(dialog), 2):
            user_tokens = tokenizer(
                f"{B_INST} {(dialog[i+1]['content']).strip()} {E_INST}",
                return_tensors="pt",
                add_special_tokens=True
            ).input_ids.to(runtimeFlag)
            assistant_w_eos = dialog[i]['content'].strip() + tokenizer.eos_token
            assistant_tokens = tokenizer(
                            assistant_w_eos,
                            return_tensors="pt",
                            add_special_tokens=False
                        ).input_ids.to(runtimeFlag)
            tokens = torch.cat([assistant_tokens, user_tokens], dim=-1)
            dialog_tokens.append(tokens)
        prompt_tokens.append(torch.cat(dialog_tokens, dim=-1))

    input_ids = prompt_tokens[0]
    if len(input_ids[0]) > max_prompt_len:
        return "\n\n **The language model's input limit has been reached. Clear the chat and start afresh!**"

    generation_output = model.generate(
        input_ids=input_ids,
        do_sample=True,
        max_new_tokens=max_gen_len,
        temperature=temperature,
        top_p=top_p,
    )

    new_tokens = generation_output[0][input_ids.shape[-1]:]
    print(tokenizer.decode(input_ids[0], skip_special_tokens=False))
    return tokenizer.decode(new_tokens, skip_special_tokens=True)


In [154]:
def print_wrapped(text, width=80):
    # Regular expression pattern to detect code blocks
    code_pattern = r'```(.+?)```'
    matches = list(re.finditer(code_pattern, text, re.DOTALL))
    if not matches:
        display(Markdown('\n'.join(textwrap.wrap(textwrap.dedent(text), width=width))))
        return

    start = 0
    for match in matches:
        before_code = text[start:match.start()].strip()
        if before_code:
            display(Markdown('\n'.join(textwrap.wrap(textwrap.dedent(before_code), width=width))))

        code = match.group(0).strip()  # Extract code block
        display(Markdown('\n' + code + '\n'))  # Display code block with space before and after

        start = match.end()

    after_code = text[start:].strip()  # Text after the last code block
    if after_code:
        display(Markdown('\n'.join(textwrap.wrap(textwrap.dedent(after_code), width=width))))

def grab_and_shorten_text(max_doc_length):

    uploaded = files.upload()

    file_name = list(uploaded.keys())[0]

    # Check the file extension
    if file_name.endswith('.txt'):
        text = uploaded[file_name].decode()
    elif file_name.endswith('.pdf'):
        pdf_bytes = io.BytesIO(uploaded[file_name])
        text = extract_text(pdf_bytes)
    else:
        raise ValueError('Unsupported file type. Please upload a .txt or .pdf file.')

    with alert_out:
        clear_output()  # Clear the previous alert
        print("Shortening the text...")

    tokens = tokenizer.encode(text, truncation=True, max_length=max_doc_length, return_tensors='pt')

    shortened_text = tokenizer.decode(tokens[0], skip_special_tokens=True)

    return file_name, shortened_text

dialog_history = [{"role": "system", "content": DEFAULT_SYSTEM_PROMPT}]

button = widgets.Button(description="Send")
upload_button = widgets.Button(description="Upload text file")
text = widgets.Textarea(layout=widgets.Layout(width='800px'))

output_log = widgets.Output()

# Define the 'Send' button click event handler
def on_button_clicked(b):
    user_input = text.value
    dialog_history.append({"role": "user", "content": user_input})

    text.value = ''

    button.description = 'Processing...'

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

    assistant_response = generate_response([dialog_history])

    button.description = 'Send'

    dialog_history.append({"role": "assistant", "content": assistant_response})

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

button.on_click(on_button_clicked)

# Create an output widget for alerts
alert_out = widgets.Output()

# Define the 'Upload' button click event handler
def on_upload_button_clicked(b):

    file_name, uploaded_text = grab_and_shorten_text(max_doc_length)

    with alert_out:
        clear_output()  # Clear the previous alert
        print(f"Upload successful: {file_name}, processing the file...")

    user_input = f"Uploaded document [{file_name}]: {uploaded_text}"
    dialog_history.append({"role": "user", "content": user_input})

    time.sleep(0.1)  # slight delay to ensure order

    assistant_input = f"You have uploaded text from {file_name}"
    dialog_history.append({"role": "assistant", "content": assistant_input})

    with output_log:
        clear_output()
        for message in dialog_history:
            print_wrapped(f'**{message["role"].capitalize()}**: {message["content"]}\n')

    with alert_out:
        clear_output()  # Clear the previous alert
        # print(f"File processing completed.")

upload_button.on_click(on_upload_button_clicked)

clear_button = widgets.Button(description="Clear Chat")
text = widgets.Textarea(layout=widgets.Layout(width='800px'))

def on_clear_button_clicked(b):
    # Clear the dialog history
    dialog_history.clear()
    # Add back the initial system prompt
    dialog_history.append({"role": "system", "content": DEFAULT_SYSTEM_PROMPT})
    # Clear the output log
    with output_log:
        clear_output()

clear_button.on_click(on_clear_button_clicked)

# Print the title
print(f"Trelis Chat (uploaded files will be shortened to {max_doc_words} words)\n" + "="*30 + "\n")

# Display the chat interface elements
display(output_log, alert_out, text, button, clear_button, upload_button)

Trelis Chat (uploaded files will be shortened to 1536 words)



Output()

Output()

Textarea(value='', layout=Layout(width='800px'))

Button(description='Send', style=ButtonStyle())

Button(description='Clear Chat', style=ButtonStyle())

Button(description='Upload text file', style=ButtonStyle())

Saving berkshire23.txt to berkshire23 (29).txt
<s> [INST] <<SYS>>
You are a helpful assistant that provides accurate and concise responses. Respond in markdown.
<</SYS>>

Uploaded document [berkshire23 (29).txt]: we are here live in Omaha Nebraska good morning everybody I'm Becky quick
along with Mike santoli and in just 30 minutes time Berkshire Hathaway chairman and CEO Warren Buffett's going to be
taking the stage with his vice chair Charlie Munger the legendary duo will also be joined by berkshire's two other
Vice chairs Greg Abel who manages the non-insurance operations for the company and Ajit Jain who runs all of the
insurance businesses and as always it's pretty big crowd here lots and lots of people and a few people you might notice
too Tim Cook is here Apple of course is still berkshire's largest holding big big part of its portfolio there you see
him backstage getting ready to go out and take his seat he gets to sit down in the special seats by the way that's
Debbie pasonic 